# Fast text approach

load libraries

In [20]:
import fasttext.util
import zipfile
import gensim
import numpy as np
import json
import random
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralCoclustering
import torch
from transformers import BertTokenizer, BertModel

load slovene BERT model

In [21]:
from transformers import CamembertTokenizer, CamembertModel
#CamembertTokenizer.from_pretrained
tokenizer = CamembertTokenizer.from_pretrained("/home/matej/Documents/FRI/NLP/NLP-tripleM/src/pretrained models/BERT/sloBERTa2/sloberta.2.0.transformers")
model = CamembertModel.from_pretrained("/home/matej/Documents/FRI/NLP/NLP-tripleM/src/pretrained models/BERT/sloBERTa2/sloberta.2.0.transformers", output_hidden_states = True)
model.eval()

Some weights of the model checkpoint at /home/matej/Documents/FRI/NLP/NLP-tripleM/src/pretrained models/BERT/sloBERTa2/sloberta.2.0.transformers were not used when initializing CamembertModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at /home/matej/Documents/FRI/NLP/NLP-tripleM/src/pretrained models/BERT/sloBERTa2/sloberta.2.0.tr

CamembertModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dr

In [22]:
text = "After stealing money from the bank vault, the bank robber was seen fishing on the Mississippi river bank."
index_of_vault = text.index("vault")
# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"
# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

print(tokenized_text)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

['▁[', 'C', 'LS', ']', '▁Af', 'ter', '▁ste', 'ali', 'ng', '▁mo', 'ney', '▁from', '▁the', '▁bank', '▁va', 'ult', ',', '▁the', '▁bank', '▁rob', 'ber', '▁was', '▁se', 'en', '▁fi', 's', 'hing', '▁on', '▁the', '▁Mi', 'ssi', 'ssi', 'p', 'pi', '▁ri', 'ver', '▁bank', '.', '▁[', 'SE', 'P', ']']
▁[              668
C            31,833
LS            7,358
]            31,858
▁Af           9,706
ter             219
▁ste            794
ali             784
ng              823
▁mo             104
ney          16,362
▁from        21,330
▁the          2,581
▁bank         5,198
▁va             316
ult          10,772
,            31,791
▁the          2,581
▁bank         5,198
▁rob         10,936
ber           1,083
▁was         31,731
▁se              49
en              410
▁fi             341
s            31,781
hing          7,768
▁on           2,708
▁the          2,581
▁Mi             537
ssi           4,046
ssi           4,046
p            31,787
pi              128
▁ri           1,216
ver          

In [23]:
segments_ids = [1] * len(tokenized_text)
print (segments_ids)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [24]:
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]


In [25]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0
print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0
print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0
print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))
print ("Number of hidden units:", hidden_states[layer_i][batch_i][token_i][0])

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 42
Number of hidden units: 768
Number of hidden units: tensor(0.2199)


In [26]:
token_vault = hidden_states[-2][0][index_of_vault]
print ("Our final sentence embedding vector of shape:", token_vault.size())

Our final sentence embedding vector of shape: torch.Size([768])


In [27]:
token_vault

tensor([-1.3619e+00, -5.4224e-01,  3.9113e-04, -1.5025e-01, -3.5773e-01,
         3.9550e-01, -4.0271e-01, -1.4918e-01, -1.7332e-01,  8.7076e-02,
        -4.5890e-01, -5.0499e-01,  3.9831e-01, -9.6095e-02,  7.4271e-03,
        -3.5406e-01,  3.0654e-01,  2.0857e-01,  4.8039e-01, -1.1335e-01,
         6.4723e-01,  1.5501e-01,  1.9120e-01,  3.6621e-01, -2.8977e-01,
         2.6011e-01,  1.9661e-01, -8.2372e-01,  1.6198e-01,  3.5347e-02,
        -1.8555e-01,  4.1205e-01, -3.5163e-01, -1.5201e-01, -4.2921e-01,
        -2.6648e-01,  1.1602e-01,  1.1035e-01, -3.3887e-01,  3.6624e-01,
        -2.5834e-01, -2.3587e-01,  8.2413e-01,  1.5979e-01, -1.3532e-02,
         1.2285e+00, -1.1470e-01,  5.4101e-01,  3.1784e-01, -8.6554e-01,
         1.2012e-01,  2.3612e-01,  4.7665e-02,  1.5308e-01,  9.1470e-02,
        -4.3538e-01,  4.3885e-01, -6.6993e-01, -6.4054e-01, -5.5695e-02,
        -3.9838e-01,  9.1519e-02,  6.8036e-01,  2.7102e-01, -2.2802e-01,
         3.9754e-01,  1.1384e+00,  1.5691e-01,  1.4